In [ ]:
import hvplot.pandas

import numpy  as np
import pandas as pd
import panel  as pn

import ipywidgets           as widgets
import matplotlib.pyplot    as plt
import plotly.graph_objects as go
import plotly.express       as px

from pathlib                   import Path
from utils.plot.total_mentions import mentions_bar
from utils.plot.stats          import stats_bar
from jupyter_bokeh             import BokehModel
from bokeh.models.formatters   import DatetimeTickFormatter


from IPython.core.display import display, HTML
from IPython.display      import (
    set_matplotlib_formats,
    clear_output,
    display
)


# display(HTML("<style>.container { width:100% !important; }</style>"))

set_matplotlib_formats('svg')

%matplotlib inline


# Sidebar

### Sidebar Card 1

"""
The sidebar is the sidebar of the dashboard.

It will always be there even after switching pages.

This content is a regular Card, for example *this* **is** [markdown](https://daringfireball.net/projects/markdown/).
"""

---

### Sidebar Card 2

In [ ]:
flex_orientation     = "row"
flex_vertical_layout = "scroll"

# Global Crypto Visualizations


# Analytics Page

## Crytpocurrency Statistics

### Crypto Price Charts (USD)

In [ ]:
tickers = ["BTC", "ETH", "ADA", "LTC", "XMR", "DOGE"]


In [ ]:
file_path = "./data/latest-quotes_data_ALL.csv"
df = pd.read_csv(file_path)
df["quote.USD.last_updated"] = pd.to_datetime(df["quote.USD.last_updated"])


#### Daily Price

In [ ]:
fig_last = df.hvplot.line(
    x='quote.USD.last_updated',
    y='quote.USD.price',
    xlabel='Date',
    ylabel='Price',
    figsize=(700,750),
    groupby='symbol',
    title='Daily Price Fluctuation',
    rot=45,
    widget_location="left",
    height=250,
    width=1350,
    fontsize={'title':'12pt','xlabel':'10pt', 'ylabel':'10pt'}
)


fig_last._sizing_mode_param_value = "stretch_both"

BokehModel(fig_last.get_root())

#### Crypto 24 Hour Rolling Average

In [ ]:
fig_24h = df.hvplot.line(
    y='quote.USD.percent_change_24h',
    x='quote.USD.last_updated',
    ylabel='Percent Change',
    xlabel='Date',
    figsize=(700,750),
    groupby='symbol',
    title='24 Hours Percent Change',
    rot=45,
    widget_location="left",
    height=250,
    width=1350,
    fontsize={'title':'12pt','xlabel':'10pt', 'ylabel':'10pt'}
)

fig_24h._sizing_mode_param_value = "stretch_both"

BokehModel(fig_24h.get_root())

#### Crypto 7 Day Rolling Average

In [ ]:
fig_7d = df.hvplot.line(
    y='quote.USD.percent_change_7d',
    x='quote.USD.last_updated',
    ylabel='Percent Change',
    xlabel='Date',
    figsize=(700,750),
    groupby='symbol',
    title='7 Days Percent Change',
    rot=45,
    widget_location="left",
    height=250,
    width=1350,
    fontsize={'title':'12pt','xlabel':'10pt', 'ylabel':'10pt'}
)

fig_7d._sizing_mode_param_value = "stretch_both"

BokehModel(fig_7d.get_root())

### Raw Price Data (USD)

In [ ]:
start_date, end_date  = 14, 19
series_time_formatter = lambda d: pd.to_datetime(d, infer_datetime_format=True).dt.strftime("%Y-%b-%d")

available_dates = [f"2021-08-{d}" for d in range(start_date, end_date)]
available_dates = list(pd.to_datetime(available_dates, format="%Y-%m-%d").strftime("%Y-%b-%d"))


In [ ]:
select_df = df.filter([
    "name",
    "last_updated",
    "quote.USD.price",
    'quote.USD.volume_24h',
    "quote.USD.percent_change_24h",
    "quote.USD.percent_change_7d",
    "quote.USD.percent_change_30d",
    "quote.USD.percent_change_90d",
])

select_df.columns = [
    "Crytpo",
    "Date",
    "Price",
    "24H Volume",
    "24H % Change",
    "7D % Change",
    "30D % Change",
    "90D % Change",
]

select_df["Date"] = pd.to_datetime(select_df["Date"], infer_datetime_format=True).dt.strftime("%d%b%y %H:%M:%S")
select_df         = select_df.sort_values(["Date", "Crytpo"], ascending=False)
select_df         = select_df.loc[series_time_formatter(select_df["Date"]).isin(available_dates)]


In [ ]:
2 * (7/2)

In [ ]:
[["rgb(31, 119, 180)", "rgb(0, 0, 0)"] * ]

In [ ]:
header_values = [f"<b>{col}</b>" for col in select_df.columns]
cell_values   = [select_df[col] for col in select_df.columns]

fig_table = go.Figure(
    data=[go.Table(
        header=dict(
            values=header_values, align='left',
            align=["left", "center"],
            font=dict(size=12),
        )
        cells=dict(values=cell_values,    align='left')
    )])

fig_table.update_layout(
    margin=go.layout.Margin(
        l=5,
        r=5,
        b=10,
        t=10,
        pad=4,
    ),
    width=1600,
)


In [ ]:
flex_orientation     = "columns"
flex_vertical_layout = "scroll"


## Media Statistics

### Media Stats

In [ ]:
# Create ipython widget.
date_stats = widgets.SelectMultiple(
    options=available_dates,
    value=available_dates,
    description="Dates:",
    disabled=False,
)

widgets.VBox([date_stats,])


In [ ]:
# Create an Ouput widget object out_stats that will hold the
# final plot.
out_stats = widgets.Output()


# Use observe() function from ipywidgets to generate the data and
# create the plot inside the output widget.
def stats_eventhandler(change):
    date           = date_stats.value
    file_path      = f"./data/ticker-stats_last7days.csv"
    df_bar         = pd.read_csv(file_path)
    df_bar["date"] = series_time_formatter(df_bar["date"])
    
    with out_stats:
        df_bar    = df_bar.loc[df_bar["date"].isin(date)]
        fig_stats = stats_bar(df_bar)

        clear_output(wait=True)        
        fig_stats.show()

date_stats.observe(stats_eventhandler, names="value")


# Output the Output widget.
stats_eventhandler(None)
out_stats



## Media Stats Col 2

### Media Sources 

In [ ]:
file_path = "./data/ticker-news_data_ALL.csv"
df = pd.read_csv(file_path)
# df.head(3)

In [ ]:
# hover is title and article
df["date"] = pd.to_datetime(df["date"])
xformatter = DatetimeTickFormatter(months='%b %Y')


fig_rank = df.hvplot.bar(
    title="Correlating News Source and Rank Score",
    y='rank_score',
    x='date',
    ylabel='Rank Score',
    xlabel='Date',
    figsize=(700,750),
    groupby='source_name',
    rot=45,
    height=500,
    widget_location="top",
    hover_cols=["title","text"],
    ylim=(3,8),
    fontsize={'title':'12pt','xlabel':'10pt', 'ylabel':'10pt'},
    xformatter=xformatter,
)

fig_rank._sizing_mode_param_value = "stretch_both"

BokehModel(fig_rank.get_root())


In [ ]:
# kind = pn.widgets.Select(
#     name='kind',
#     value='bar',
#     options=['bar', 'line','scatter']
# )

# fig_type = df.hvplot(
#     x= 'date',
#     y='rank_score',
#     ylabel='Rank Score',
#     kind=kind,
#     colorbar=False,
#     width=900,
#     widget_location="bottom",
#     xaxis="",
#     hover_cols=["title","text"],
#     title="Various Views of Rank Score"
# )


# BokehModel(fig_type.get_root())